In [6]:
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from FaceBinaryDataset import FaceBinaryDataset
from build_pipeline import build_pipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dataset = FaceBinaryDataset("dataset/output_dataset", dimension=160)

images = []
labels = []

for i in range(len(dataset)):
    img, label, userid = dataset[i]

    if img is None:
        continue

    images.append(img)
    labels.append(label)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

No face detected in: dataset/output_dataset/authorized/user_00041/img_015.jpg
No face detected in: dataset/output_dataset/authorized/user_00041/img_013.jpg
No face detected in: dataset/output_dataset/authorized/user_00041/img_008.jpg
No face detected in: dataset/output_dataset/authorized/user_00079/img_003.jpg
No face detected in: dataset/output_dataset/authorized/user_00084/img_003.jpg
No face detected in: dataset/output_dataset/authorized/user_00084/img_011.jpg
No face detected in: dataset/output_dataset/authorized/user_00084/img_020.jpg
No face detected in: dataset/output_dataset/authorized/user_00070/img_020.jpg
No face detected in: dataset/output_dataset/authorized/user_00070/img_019.jpg
No face detected in: dataset/output_dataset/authorized/user_00077/img_003.jpg
No face detected in: dataset/output_dataset/authorized/user_00077/img_006.jpg
No face detected in: dataset/output_dataset/authorized/user_00083/img_012.jpg
No face detected in: dataset/output_dataset/authorized/user_0008

In [12]:
for name in [ "cnn","facenet","lbp", "hog"]:
    print(f"{name.upper()}")
    pipeline = build_pipeline(name)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred))


CNN
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━

/Users/dawid/PycharmProjects/ImbalancedDataProject/.venv/lib/python3.9/site-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


KeyboardInterrupt: 